In [0]:

from pyspark.sql import functions as F

d1 = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_back/dimension/IOS_COUNTRY_MAPPING",sep="\t").withColumnRenamed("_c0","store_id").withColumnRenamed("_c1","country_code").withColumn("market_code",F.lit("ios"))
d2 = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_back/dimension/ANDROID_COUNTRY_MAPPING",sep="\t").withColumnRenamed("_c0","store_id").withColumnRenamed("_c1","country_code").withColumn("market_code",F.lit("android")).cache()


df2.filter("category in (17, 56, 62)").show()


In [0]:
%%sh
# aws s3 ls s3://b2c-prod-dca-store-estimates/store_back/dimension/IOS_COUNTRY_MAPPING/
# # aws s3 cp s3://b2c-prod-data-pipeline-qa/aa.store/store_cateogry_mapping/  s3://b2c-prod-data-pipeline-unified-qa/obselete/store/store_cateogry_mapping/ --recursive
# aws s3 cp s3://b2c-prod-dca-store-estimates/store_back/dimension/IOS_COUNTRY_MAPPING/  s3://b2c-prod-data-pipeline-unified-qa/obselete/store/dimension/IOS_COUNTRY_MAPPING --recursive
# aws s3 cp s3://b2c-prod-dca-store-estimates/store_back/dimension/ANDROID_COUNTRY_MAPPING/  s3://b2c-prod-data-pipeline-unified-qa/obselete/store/dimension/ANDROID_COUNTRY_MAPPING --recursive

aws s3 ls s3://b2c-prod-dca-store-estimates/store_back/category-uniform/category_code/
 
 


In [0]:

cdf_service= spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/store_cateogry_mapping").cache()
cdf_datafoundation= spark.read.csv("s3://b2c-prod-dca-store-estimates/store_back/category-uniform/category_meta/",sep="\t").cache()

cdf_datafoundation.createOrReplaceTempView("temp")
spark.sql("select * from temp where _c1=6000").show()
# compared with service deminsio
# cdf_service.join(cdf_datafoundation, cdf_service.legacy_category_id==cdf_datafoundation._c1,"outer").filter(" _c1 is null").show()
# print cdf_service.join(cdf_datafoundation, cdf_service.legacy_category_id==cdf_datafoundation._c1,"outer").filter("_c1 is null").select("legacy_category_id","category_id").collect()



# # compared with auto pipeline
# print cdf_datafoundation.join(temp_df, [cdf_datafoundation._c1==temp_df.auto_pipeline_legacy_category_id,  cdf_datafoundation._c0==temp_df.auto_pipeline_market], "outer" ).select("auto_pipeline_legacy_category_id","auto_pipeline_category_id").filter("_c1 is null").collect()
# cdf_datafoundation.join(temp_df, [cdf_datafoundation._c1==temp_df.auto_pipeline_legacy_category_id,  cdf_datafoundation._c0==temp_df.auto_pipeline_market], "outer" ).filter("auto_pipeline_legacy_category_id is null and _c0 != 'mac'").show(10)



In [0]:

from pyspark.sql import functions as F

d1 = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_back/dimension/IOS_COUNTRY_MAPPING",sep="\t").withColumnRenamed("_c0","store_id").withColumnRenamed("_c1","country_code").withColumn("market_code",F.lit("ios"))
d2 = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_back/dimension/ANDROID_COUNTRY_MAPPING",sep="\t").withColumnRenamed("_c0","store_id").withColumnRenamed("_c1","country_code").withColumn("market_code",F.lit("android"))

country_code_df = d1.union(d2).cache()
country_code_df.show(2)

country_code_service_df = spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/country_mapping").withColumnRenamed("market_code","market_code_service").cache()

new_country_code_df = country_code_df.withColumn("market_code_new", F.when(country_code_df["market_code"]=="ios","apple-store").when(country_code_df["market_code"]=="android","google-play").otherwise(country_code_df["market_code"])).cache()

# check deminsion service : 
join_condition = [new_country_code_df.store_id==country_code_service_df.country_id , new_country_code_df.market_code_new==country_code_service_df.market_code_service ]
join_result = new_country_code_df.join(country_code_service_df,join_condition ,"outer").filter("market_code_service in ('apple-store' , 'google-play')").cache()
join_result.show(2)
join_result.drop("market_code","market_code_new").filter("market_code_service='google-play'").filter("store_id is null").show()
join_result.drop("market_code","market_code_new").filter("market_code_service='apple-store'").filter("store_id is null").show()
join_result.drop("market_code","market_code_new").filter("market_code_service='google-play'").filter("country_id is null").show()
join_result.drop("market_code","market_code_new").filter("market_code_service='apple-store'").filter("country_id is null").show()




# temp_df.join(new_country_code_df,[new_country_code_df.store_id==temp_df.auto_pipeline_id, new_country_code_df.market_code_new==temp_df.auto_pipeline_market], "outer").filter("market_code_new='google-play' and auto_pipeline_id is null").show()
# temp_df.join(new_country_code_df,[new_country_code_df.store_id==temp_df.auto_pipeline_id, new_country_code_df.market_code_new==temp_df.auto_pipeline_market], "outer").filter("market_code_new='apple-store' and auto_pipeline_id is null").show()
# temp_df.join(new_country_code_df,[new_country_code_df.store_id==temp_df.auto_pipeline_id, new_country_code_df.market_code_new==temp_df.auto_pipeline_market], "outer").filter("market_code_new='google-play' and store_id is null").show()
# temp_df.join(new_country_code_df,[new_country_code_df.store_id==temp_df.auto_pipeline_id, new_country_code_df.market_code_new==temp_df.auto_pipeline_market], "outer").filter("market_code_new='apple-store' and  is null").show()

auto_pipeline_test_df = temp_df.join(new_country_code_df,[new_country_code_df.store_id==temp_df.auto_pipeline_id, new_country_code_df.market_code_new==temp_df.auto_pipeline_market], "outer").filter("market_code_new='google-play' and auto_pipeline_id is null").collect()
# auto_pipeline_test_df = temp_df.join(new_country_code_df,[new_country_code_df.store_id==temp_df.auto_pipeline_id, new_country_code_df.market_code_new==temp_df.auto_pipeline_market], "outer").filter("market_code_new='google-play' and auto_pipeline_id is null").collect()

print auto_pipeline_test_df

for x in auto_pipeline_test_df:
    print x["store_id"], x["country_code"],x["market_code"]



In [0]:

spark.read.csv("s3://b2c-prod-dca-store-estimates/store_back/dimension/ANDROID_COUNTRY_MAPPING",sep="\t").where("_c1='WW'").show()

In [0]:

country_code_list = [(u'apple-store', 143441, u'US'), (u'google-play', 10, u'US'), (u'amazon-store', 1, u'US'), (u'windows-store', 1, u'US'), (u'windows-phone', 1, u'US'), (u'appannie', 10, u'US'), (u'apple-store', 143442, u'FR'), (u'google-play', 6, u'FR'), (u'amazon-store', 4, u'FR'), (u'windows-store', 5, u'FR'), (u'windows-phone', 14, u'FR'), (u'appannie', 6, u'FR'), (u'apple-store', 143443, u'DE'), (u'google-play', 4, u'DE'), (u'amazon-store', 2, u'DE'), (u'windows-store', 4, u'DE'), (u'windows-phone', 12, u'DE'), (u'appannie', 4, u'DE'), (u'apple-store', 143444, u'GB'), (u'google-play', 7, u'GB'), (u'windows-store', 6, u'GB'), (u'windows-phone', 34, u'GB'), (u'appannie', 7, u'GB'), (u'apple-store', 143445, u'AT'), (u'google-play', 35, u'AT'), (u'windows-store', 44, u'AT'), (u'windows-phone', 25, u'AT'), (u'appannie', 35, u'AT'), (u'apple-store', 143446, u'BE'), (u'google-play', 11, u'BE'), (u'windows-store', 12, u'BE'), (u'windows-phone', 4, u'BE'), (u'appannie', 11, u'BE'), (u'apple-store', 143447, u'FI'), (u'google-play', 20, u'FI'), (u'windows-store', 17, u'FI'), (u'windows-phone', 32, u'FI'), (u'appannie', 20, u'FI'), (u'apple-store', 143448, u'GR'), (u'google-play', 46, u'GR'), (u'windows-phone', 35, u'GR'), (u'appannie', 46, u'GR'), (u'apple-store', 143449, u'IE'), (u'google-play', 39, u'IE'), (u'windows-store', 21, u'IE'), (u'windows-phone', 18, u'IE'), (u'appannie', 39, u'IE'), (u'apple-store', 143450, u'IT'), (u'google-play', 8, u'IT'), (u'amazon-store', 5, u'IT'), (u'windows-store', 23, u'IT'), (u'windows-phone', 19, u'IT'), (u'appannie', 8, u'IT'), (u'apple-store', 143451, u'LU'), (u'google-play', 103, u'LU'), (u'windows-store', 67, u'LU'), (u'windows-phone', 76, u'LU'), (u'apple-store', 143452, u'NL'), (u'google-play', 23, u'NL'), (u'windows-store', 26, u'NL'), (u'windows-phone', 22, u'NL'), (u'appannie', 23, u'NL'), (u'apple-store', 143453, u'PT'), (u'google-play', 43, u'PT'), (u'windows-store', 31, u'PT'), (u'windows-phone', 27, u'PT'), (u'appannie', 43, u'PT'), (u'apple-store', 143454, u'ES'), (u'google-play', 5, u'ES'), (u'amazon-store', 3, u'ES'), (u'windows-store', 35, u'ES'), (u'windows-phone', 13, u'ES'), (u'appannie', 5, u'ES'), (u'apple-store', 143455, u'CA'), (u'google-play', 2, u'CA'), (u'amazon-store', 10, u'CA'), (u'windows-store', 3, u'CA'), (u'windows-phone', 6, u'CA'), (u'appannie', 2, u'CA'), (u'apple-store', 143456, u'SE'), (u'google-play', 34, u'SE'), (u'windows-store', 36, u'SE'), (u'windows-phone', 33, u'SE'), (u'appannie', 34, u'SE'), (u'apple-store', 143457, u'NO'), (u'google-play', 42, u'NO'), (u'windows-store', 28, u'NO'), (u'windows-phone', 24, u'NO'), (u'appannie', 42, u'NO'), (u'apple-store', 143458, u'DK'), (u'google-play', 38, u'DK'), (u'windows-store', 16, u'DK'), (u'windows-phone', 11, u'DK'), (u'appannie', 38, u'DK'), (u'apple-store', 143459, u'CH'), (u'google-play', 12, u'CH'), (u'windows-store', 41, u'CH'), (u'windows-phone', 29, u'CH'), (u'appannie', 12, u'CH'), (u'apple-store', 143460, u'AU'), (u'google-play', 1, u'AU'), (u'amazon-store', 13, u'AU'), (u'windows-store', 2, u'AU'), (u'windows-phone', 2, u'AU'), (u'appannie', 1, u'AU'), (u'apple-store', 143461, u'NZ'), (u'google-play', 41, u'NZ'), (u'windows-store', 27, u'NZ'), (u'windows-phone', 23, u'NZ'), (u'appannie', 41, u'NZ'), (u'apple-store', 143462, u'JP'), (u'google-play', 9, u'JP'), (u'amazon-store', 6, u'JP'), (u'windows-store', 7, u'JP'), (u'windows-phone', 36, u'JP'), (u'appannie', 9, u'JP'), (u'apple-store', 143463, u'HK'), (u'google-play', 16, u'HK'), (u'windows-store', 18, u'HK'), (u'windows-phone', 15, u'HK'), (u'appannie', 16, u'HK'), (u'apple-store', 143464, u'SG'), (u'google-play', 32, u'SG'), (u'windows-store', 33, u'SG'), (u'windows-phone', 30, u'SG'), (u'appannie', 32, u'SG'), (u'apple-store', 143465, u'CN'), (u'google-play', 3, u'CN'), (u'amazon-store', 8, u'CN'), (u'windows-store', 10, u'CN'), (u'windows-phone', 40, u'CN'), (u'appannie', 3, u'CN'), (u'apple-store', 143466, u'KR'), (u'google-play', 27, u'KR'), (u'windows-store', 8, u'KR'), (u'windows-phone', 37, u'KR'), (u'appannie', 27, u'KR'), (u'apple-store', 143467, u'IN'), (u'google-play', 19, u'IN'), (u'amazon-store', 11, u'IN'), (u'windows-store', 19, u'IN'), (u'windows-phone', 17, u'IN'), (u'appannie', 19, u'IN'), (u'apple-store', 143468, u'MX'), (u'google-play', 26, u'MX'), (u'amazon-store', 12, u'MX'), (u'windows-store', 25, u'MX'), (u'windows-phone', 21, u'MX'), (u'appannie', 26, u'MX'), (u'apple-store', 143469, u'RU'), (u'google-play', 22, u'RU'), (u'windows-store', 9, u'RU'), (u'windows-phone', 38, u'RU'), (u'appannie', 22, u'RU'), (u'apple-store', 143470, u'TW'), (u'google-play', 30, u'TW'), (u'windows-store', 37, u'TW'), (u'windows-phone', 39, u'TW'), (u'appannie', 30, u'TW'), (u'apple-store', 143471, u'VN'), (u'google-play', 15, u'VN'), (u'windows-store', 40, u'VN'), (u'windows-phone', 66, u'VN'), (u'appannie', 15, u'VN'), (u'apple-store', 143472, u'ZA'), (u'google-play', 14, u'ZA'), (u'windows-store', 34, u'ZA'), (u'windows-phone', 31, u'ZA'), (u'appannie', 14, u'ZA'), (u'apple-store', 143473, u'MY'), (u'google-play', 24, u'MY'), (u'windows-store', 24, u'MY'), (u'windows-phone', 50, u'MY'), (u'appannie', 24, u'MY'), (u'apple-store', 143474, u'PH'), (u'google-play', 31, u'PH'), (u'windows-store', 29, u'PH'), (u'windows-phone', 52, u'PH'), (u'appannie', 31, u'PH'), (u'apple-store', 143475, u'TH'), (u'google-play', 29, u'TH'), (u'windows-store', 38, u'TH'), (u'windows-phone', 64, u'TH'), (u'appannie', 29, u'TH'), (u'apple-store', 143476, u'ID'), (u'google-play', 21, u'ID'), (u'windows-store', 20, u'ID'), (u'windows-phone', 47, u'ID'), (u'appannie', 21, u'ID'), (u'apple-store', 143477, u'PK'), (u'google-play', 54, u'PK'), (u'windows-store', 81, u'PK'), (u'windows-phone', 166, u'PK'), (u'appannie', 54, u'PK'), (u'apple-store', 143478, u'PL'), (u'google-play', 28, u'PL'), (u'windows-store', 30, u'PL'), (u'windows-phone', 26, u'PL'), (u'appannie', 28, u'PL'), (u'apple-store', 143479, u'SA'), (u'google-play', 51, u'SA'), (u'windows-phone', 63, u'SA'), (u'appannie', 51, u'SA'), (u'apple-store', 143480, u'TR'), (u'google-play', 25, u'TR'), (u'windows-store', 39, u'TR'), (u'windows-phone', 56, u'TR'), (u'appannie', 25, u'TR'), (u'apple-store', 143481, u'AE'), (u'google-play', 49, u'AE'), (u'windows-store', 77, u'AE'), (u'windows-phone', 65, u'AE'), (u'appannie', 49, u'AE'), (u'apple-store', 143482, u'HU'), (u'google-play', 37, u'HU'), (u'windows-store', 43, u'HU'), (u'windows-phone', 20, u'HU'), (u'appannie', 37, u'HU'), (u'apple-store', 143483, u'CL'), (u'google-play', 13, u'CL'), (u'windows-store', 14, u'CL'), (u'windows-phone', 8, u'CL'), (u'appannie', 13, u'CL'), (u'apple-store', 143484, u'NP'), (u'google-play', 91, u'NP'), (u'windows-phone', 162, u'NP'), (u'apple-store', 143485, u'PA'), (u'google-play', 77, u'PA'), (u'windows-store', 69, u'PA'), (u'windows-phone', 77, u'PA'), (u'apple-store', 143486, u'LK'), (u'google-play', 82, u'LK'), (u'windows-phone', 138, u'LK'), (u'apple-store', 143487, u'RO'), (u'google-play', 44, u'RO'), (u'windows-store', 32, u'RO'), (u'windows-phone', 53, u'RO'), (u'appannie', 44, u'RO'), (u'apple-store', 143489, u'CZ'), (u'google-play', 36, u'CZ'), (u'windows-store', 15, u'CZ'), (u'windows-phone', 10, u'CZ'), (u'appannie', 36, u'CZ'), (u'apple-store', 143491, u'IL'), (u'google-play', 40, u'IL'), (u'windows-store', 22, u'IL'), (u'windows-phone', 61, u'IL'), (u'appannie', 40, u'IL'), (u'apple-store', 143492, u'UA'), (u'google-play', 48, u'UA'), (u'windows-store', 76, u'UA'), (u'windows-phone', 58, u'UA'), (u'appannie', 48, u'UA'), (u'apple-store', 143493, u'KW'), (u'google-play', 50, u'KW'), (u'windows-phone', 133, u'KW'), (u'appannie', 50, u'KW'), (u'apple-store', 143494, u'HR'), (u'google-play', 80, u'HR'), (u'windows-store', 55, u'HR'), (u'windows-phone', 44, u'HR'), (u'appannie', 80, u'HR'), (u'apple-store', 143495, u'CR'), (u'google-play', 64, u'CR'), (u'windows-store', 53, u'CR'), (u'windows-phone', 43, u'CR'), (u'appannie', 64, u'CR'), (u'apple-store', 143496, u'SK'), (u'google-play', 45, u'SK'), (u'windows-store', 73, u'SK'), (u'windows-phone', 54, u'SK'), (u'appannie', 45, u'SK'), (u'apple-store', 143497, u'LB'), (u'google-play', 65, u'LB'), (u'windows-phone', 135, u'LB'), (u'appannie', 65, u'LB'), (u'apple-store', 143498, u'QA'), (u'google-play', 73, u'QA'), (u'windows-store', 71, u'QA'), (u'windows-phone', 62, u'QA'), (u'apple-store', 143499, u'SI'), (u'google-play', 96, u'SI'), (u'windows-store', 74, u'SI'), (u'windows-phone', 55, u'SI'), (u'apple-store', 143501, u'CO'), (u'google-play', 52, u'CO'), (u'windows-store', 52, u'CO'), (u'windows-phone', 9, u'CO'), (u'appannie', 52, u'CO'), (u'apple-store', 143502, u'VE'), (u'google-play', 60, u'VE'), (u'windows-store', 79, u'VE'), (u'windows-phone', 57, u'VE'), (u'apple-store', 143503, u'BR'), (u'google-play', 18, u'BR'), (u'amazon-store', 9, u'BR'), (u'windows-store', 13, u'BR'), (u'windows-phone', 5, u'BR'), (u'appannie', 18, u'BR'), (u'apple-store', 143504, u'GT'), (u'google-play', 67, u'GT'), (u'windows-phone', 122, u'GT'), (u'apple-store', 143505, u'AR'), (u'google-play', 17, u'AR'), (u'windows-store', 11, u'AR'), (u'windows-phone', 41, u'AR'), (u'appannie', 17, u'AR'), (u'apple-store', 143506, u'SV'), (u'google-play', 75, u'SV'), (u'windows-phone', 177, u'SV'), (u'apple-store', 143507, u'PE'), (u'google-play', 56, u'PE'), (u'windows-store', 70, u'PE'), (u'windows-phone', 51, u'PE'), (u'appannie', 56, u'PE'), (u'apple-store', 143508, u'DO'), (u'google-play', 69, u'DO'), (u'windows-phone', 108, u'DO'), (u'apple-store', 143509, u'EC'), (u'google-play', 62, u'EC'), (u'windows-store', 57, u'EC'), (u'windows-phone', 71, u'EC'), (u'apple-store', 143510, u'HN'), (u'google-play', 83, u'HN'), (u'windows-phone', 125, u'HN'), (u'apple-store', 143511, u'JM'), (u'google-play', 81, u'JM'), (u'windows-phone', 127, u'JM'), (u'apple-store', 143512, u'NI'), (u'windows-store', 68, u'NI'), (u'windows-phone', 161, u'NI'), (u'apple-store', 143513, u'PY'), (u'google-play', 89, u'PY'), (u'windows-phone', 168, u'PY'), (u'apple-store', 143514, u'UY'), (u'google-play', 85, u'UY'), (u'windows-phone', 81, u'UY'), (u'apple-store', 143515, u'MO'), (u'google-play', 102, u'MO'), (u'windows-phone', 151, u'MO'), (u'apple-store', 143516, u'EG'), (u'google-play', 33, u'EG'), (u'windows-store', 42, u'EG'), (u'windows-phone', 72, u'EG'), (u'appannie', 33, u'EG'), (u'apple-store', 143517, u'KZ'), (u'google-play', 53, u'KZ'), (u'windows-store', 63, u'KZ'), (u'windows-phone', 67, u'KZ'), (u'appannie', 53, u'KZ'), (u'apple-store', 143518, u'EE'), (u'windows-phone', 45, u'EE'), (u'apple-store', 143519, u'LV'), (u'google-play', 86, u'LV'), (u'windows-store', 65, u'LV'), (u'windows-phone', 48, u'LV'), (u'appannie', 86, u'LV'), (u'apple-store', 143520, u'LT'), (u'google-play', 78, u'LT'), (u'windows-store', 66, u'LT'), (u'windows-phone', 49, u'LT'), (u'appannie', 78, u'LT'), (u'apple-store', 143521, u'MT'), (u'windows-phone', 153, u'MT'), (u'apple-store', 143523, u'MD'), (u'google-play', 90, u'MD'), (u'apple-store', 143524, u'AM'), (u'google-play', 97, u'AM'), (u'windows-phone', 85, u'AM'), (u'apple-store', 143525, u'BW'), (u'windows-phone', 98, u'BW'), (u'apple-store', 143526, u'BG'), (u'google-play', 47, u'BG'), (u'windows-store', 49, u'BG'), (u'windows-phone', 42, u'BG'), (u'appannie', 47, u'BG'), (u'apple-store', 143528, u'JO'), (u'google-play', 63, u'JO'), (u'windows-store', 62, u'JO'), (u'windows-phone', 74, u'JO'), (u'apple-store', 143529, u'KE'), (u'google-play', 95, u'KE'), (u'windows-store', 64, u'KE'), (u'windows-phone', 75, u'KE'), (u'appannie', 95, u'KE'), (u'apple-store', 143530, u'MK'), (u'google-play', 100, u'MK'), (u'windows-phone', 147, u'MK'), (u'apple-store', 143531, u'MG'), (u'windows-phone', 145, u'MG'), (u'apple-store', 143532, u'ML'), (u'windows-phone', 148, u'ML'), (u'apple-store', 143533, u'MU'), (u'windows-phone', 154, u'MU'), (u'apple-store', 143534, u'NE'), (u'windows-phone', 159, u'NE'), (u'apple-store', 143535, u'SN'), (u'windows-store', 72, u'SN'), (u'windows-phone', 78, u'SN'), (u'apple-store', 143536, u'TN'), (u'google-play', 79, u'TN'), (u'windows-store', 75, u'TN'), (u'windows-phone', 80, u'TN'), (u'apple-store', 143537, u'UG'), (u'windows-phone', 188, u'UG'), (u'apple-store', 143538, u'AI'), (u'apple-store', 143539, u'BS'), (u'windows-store', 46, u'BS'), (u'windows-phone', 96, u'BS'), (u'apple-store', 143540, u'AG'), (u'windows-store', 80, u'AG'), (u'windows-phone', 68, u'AG'), (u'apple-store', 143541, u'BB'), (u'windows-store', 48, u'BB'), (u'windows-phone', 89, u'BB'), (u'apple-store', 143542, u'BM'), (u'apple-store', 143543, u'VG'), (u'apple-store', 143544, u'KY'), (u'windows-store', 51, u'KY'), (u'apple-store', 143545, u'DM'), (u'windows-phone', 107, u'DM'), (u'apple-store', 143546, u'GD'), (u'windows-phone', 116, u'GD'), (u'apple-store', 143547, u'MS'), (u'apple-store', 143548, u'KN'), (u'windows-phone', 132, u'KN'), (u'apple-store', 143549, u'LC'), (u'windows-phone', 136, u'LC'), (u'apple-store', 143550, u'VC'), (u'windows-phone', 190, u'VC'), (u'apple-store', 143551, u'TT'), (u'google-play', 99, u'TT'), (u'windows-phone', 185, u'TT'), (u'apple-store', 143552, u'TC'), (u'apple-store', 143553, u'GY'), (u'windows-phone', 124, u'GY'), (u'apple-store', 143554, u'SR'), (u'windows-phone', 175, u'SR'), (u'apple-store', 143555, u'BZ'), (u'windows-phone', 100, u'BZ'), (u'apple-store', 143556, u'BO'), (u'google-play', 72, u'BO'), (u'windows-phone', 95, u'BO'), (u'apple-store', 143557, u'CY'), (u'windows-store', 56, u'CY'), (u'windows-phone', 70, u'CY'), (u'apple-store', 143558, u'IS'), (u'windows-store', 60, u'IS'), (u'windows-phone', 46, u'IS'), (u'apple-store', 143559, u'BH'), (u'google-play', 93, u'BH'), (u'windows-store', 47, u'BH'), (u'windows-phone', 59, u'BH'), (u'apple-store', 143560, u'BN'), (u'windows-phone', 94, u'BN'), (u'apple-store', 143561, u'NG'), (u'google-play', 74, u'NG'), (u'windows-phone', 160, u'NG'), (u'appannie', 74, u'NG'), (u'apple-store', 143562, u'OM'), (u'google-play', 71, u'OM'), (u'windows-phone', 164, u'OM'), (u'apple-store', 143563, u'DZ'), (u'google-play', 59, u'DZ'), (u'windows-phone', 109, u'DZ'), (u'apple-store', 143564, u'AO'), (u'windows-phone', 86, u'AO'), (u'apple-store', 143565, u'BY'), (u'google-play', 58, u'BY'), (u'windows-phone', 99, u'BY'), (u'apple-store', 143566, u'UZ'), (u'google-play', 98, u'UZ'), (u'windows-phone', 189, u'UZ'), (u'apple-store', 143568, u'AZ'), (u'google-play', 61, u'AZ'), (u'windows-phone', 87, u'AZ'), (u'appannie', 61, u'AZ'), (u'apple-store', 143571, u'YE'), (u'google-play', 101, u'YE'), (u'windows-phone', 193, u'YE'), (u'apple-store', 143572, u'TZ'), (u'google-play', 92, u'TZ'), (u'windows-phone', 187, u'TZ'), (u'apple-store', 143573, u'GH'), (u'google-play', 94, u'GH'), (u'windows-store', 59, u'GH'), (u'windows-phone', 73, u'GH'), (u'apple-store', 143575, u'AL'), (u'windows-phone', 84, u'AL'), (u'apple-store', 143576, u'BJ'), (u'windows-phone', 93, u'BJ'), (u'apple-store', 143577, u'BT'), (u'windows-phone', 97, u'BT'), (u'apple-store', 143578, u'BF'), (u'windows-phone', 91, u'BF'), (u'apple-store', 143579, u'KH'), (u'google-play', 76, u'KH'), (u'windows-phone', 129, u'KH'), (u'apple-store', 143580, u'CV'), (u'windows-phone', 105, u'CV'), (u'apple-store', 143581, u'TD'), (u'windows-phone', 179, u'TD'), (u'apple-store', 143582, u'CG'), (u'windows-phone', 103, u'CG'), (u'apple-store', 143583, u'FJ'), (u'windows-phone', 112, u'FJ'), (u'apple-store', 143584, u'GM'), (u'windows-phone', 119, u'GM'), (u'apple-store', 143585, u'GW'), (u'windows-phone', 123, u'GW'), (u'apple-store', 143586, u'KG'), (u'google-play', 88, u'KG'), (u'windows-phone', 128, u'KG'), (u'apple-store', 143587, u'LA'), (u'windows-phone', 134, u'LA'), (u'apple-store', 143588, u'LR'), (u'windows-phone', 139, u'LR'), (u'apple-store', 143589, u'MW'), (u'windows-phone', 156, u'MW'), (u'apple-store', 143590, u'MR'), (u'windows-phone', 152, u'MR'), (u'apple-store', 143591, u'FM'), (u'windows-phone', 113, u'FM'), (u'apple-store', 143592, u'MN'), (u'windows-phone', 150, u'MN'), (u'apple-store', 143593, u'MZ'), (u'windows-phone', 157, u'MZ'), (u'apple-store', 143594, u'NA'), (u'windows-phone', 158, u'NA'), (u'apple-store', 143595, u'PW'), (u'windows-phone', 167, u'PW'), (u'apple-store', 143597, u'PG'), (u'windows-phone', 165, u'PG'), (u'apple-store', 143598, u'ST'), (u'windows-phone', 176, u'ST'), (u'apple-store', 143599, u'SC'), (u'windows-phone', 172, u'SC'), (u'apple-store', 143600, u'SL'), (u'windows-phone', 79, u'SL'), (u'apple-store', 143601, u'SB'), (u'windows-phone', 171, u'SB'), (u'apple-store', 143602, u'SZ'), (u'windows-phone', 178, u'SZ'), (u'apple-store', 143603, u'TJ'), (u'windows-phone', 181, u'TJ'), (u'apple-store', 143604, u'TM'), (u'windows-phone', 183, u'TM'), (u'apple-store', 143605, u'ZW'), (u'windows-phone', 195, u'ZW'), (u'google-play', 55, u'IQ'), (u'windows-store', 61, u'IQ'), (u'windows-phone', 60, u'IQ'), (u'google-play', 57, u'MA'), (u'windows-phone', 142, u'MA'), (u'google-play', 66, u'BD'), (u'windows-phone', 90, u'BD'), (u'google-play', 68, u'RS'), (u'windows-store', 82, u'RS'), (u'windows-phone', 169, u'RS'), (u'google-play', 70, u'IR'), (u'google-play', 84, u'PR'), (u'google-play', 87, u'BA'), (u'windows-phone', 88, u'BA'), (u'amazon-store', 7, u'UK'), (u'windows-store', 50, u'CM'), (u'windows-phone', 69, u'CM'), (u'windows-store', 54, u'CI'), (u'windows-phone', 104, u'CI'), (u'windows-store', 78, u'UM'), (u'windows-phone', 82, u'AD'), (u'windows-phone', 83, u'AF'), (u'windows-phone', 92, u'BI'), (u'windows-phone', 101, u'CD'), (u'windows-phone', 102, u'CF'), (u'windows-phone', 106, u'DJ'), (u'windows-phone', 110, u'ER'), (u'windows-phone', 111, u'ET'), (u'windows-phone', 114, u'FO'), (u'windows-phone', 115, u'GA'), (u'windows-phone', 117, u'GE'), (u'windows-phone', 118, u'GL'), (u'windows-phone', 120, u'GN'), (u'windows-phone', 121, u'GQ'), (u'windows-phone', 126, u'HT'), (u'windows-phone', 130, u'KI'), (u'windows-phone', 131, u'KM'), (u'windows-phone', 137, u'LI'), (u'windows-phone', 140, u'LS'), (u'windows-phone', 141, u'LY'), (u'windows-phone', 143, u'MC'), (u'windows-phone', 144, u'ME'), (u'windows-phone', 146, u'MH'), (u'windows-phone', 149, u'MM'), (u'windows-phone', 155, u'MV'), (u'windows-phone', 163, u'NR'), (u'windows-phone', 170, u'RW'), (u'windows-phone', 173, u'SM'), (u'windows-phone', 174, u'SO'), (u'windows-phone', 180, u'TG'), (u'windows-phone', 182, u'TL'), (u'windows-phone', 184, u'TO'), (u'windows-phone', 186, u'TV'), (u'windows-phone', 191, u'VU'), (u'windows-phone', 192, u'WS'), (u'windows-phone', 194, u'ZM'), (u'appannie', None, u'WW'), ('apple-store', 0, 'WW'), ('google-play', 1000, 'WW')]


spark.createDataFrame(country_code_list, ["market_code","country_id","country_code"]).coalesce(1).write.format("delta").mode("overwrite").save("s3://b2c-prod-data-pipeline-qa/aa.store/country_mapping/")



In [0]:

IOS_STORE_COUNTRY_MAPPING = [
    (0, 'WW'), (143575, 'AL'), (143563, 'DZ'), (143564, 'AO'), (143538, 'AI'),
    (143540, 'AG'), (143505, 'AR'), (143524, 'AM'), (143460, 'AU'), (143445, 'AT'),
    (143568, 'AZ'), (143539, 'BS'), (143559, 'BH'), (143541, 'BB'), (143565, 'BY'),
    (143446, 'BE'), (143555, 'BZ'), (143576, 'BJ'), (143542, 'BM'), (143577, 'BT'),
    (143556, 'BO'), (143525, 'BW'), (143503, 'BR'), (143543, 'VG'), (143560, 'BN'),
    (143526, 'BG'), (143578, 'BF'), (143579, 'KH'), (143455, 'CA'), (143580, 'CV'),
    (143544, 'KY'), (143581, 'TD'), (143483, 'CL'), (143465, 'CN'), (143501, 'CO'),
    (143582, 'CG'), (143495, 'CR'), (143494, 'HR'), (143557, 'CY'), (143489, 'CZ'),
    (143458, 'DK'), (143545, 'DM'), (143508, 'DO'), (143509, 'EC'), (143516, 'EG'),
    (143506, 'SV'), (143518, 'EE'), (143583, 'FJ'), (143447, 'FI'), (143442, 'FR'),
    (143584, 'GM'), (143443, 'DE'), (143573, 'GH'), (143448, 'GR'), (143546, 'GD'),
    (143504, 'GT'), (143585, 'GW'), (143553, 'GY'), (143510, 'HN'), (143463, 'HK'),
    (143482, 'HU'), (143558, 'IS'), (143467, 'IN'), (143476, 'ID'), (143449, 'IE'),
    (143491, 'IL'), (143450, 'IT'), (143511, 'JM'), (143462, 'JP'), (143528, 'JO'),
    (143517, 'KZ'), (143529, 'KE'), (143493, 'KW'), (143586, 'KG'), (143587, 'LA'),
    (143519, 'LV'), (143497, 'LB'), (143588, 'LR'), (143520, 'LT'), (143451, 'LU'),
    (143515, 'MO'), (143530, 'MK'), (143531, 'MG'), (143589, 'MW'), (143473, 'MY'),
    (143532, 'ML'), (143521, 'MT'), (143590, 'MR'), (143533, 'MU'), (143468, 'MX'),
    (143591, 'FM'), (143523, 'MD'), (143592, 'MN'), (143547, 'MS'), (143593, 'MZ'),
    (143594, 'NA'), (143484, 'NP'), (143452, 'NL'), (143461, 'NZ'), (143512, 'NI'),
    (143534, 'NE'), (143561, 'NG'), (143457, 'NO'), (143562, 'OM'), (143477, 'PK'),
    (143595, 'PW'), (143485, 'PA'), (143597, 'PG'), (143513, 'PY'), (143507, 'PE'),
    (143474, 'PH'), (143478, 'PL'), (143453, 'PT'), (143498, 'QA'), (143487, 'RO'),
    (143469, 'RU'), (143598, 'ST'), (143479, 'SA'), (143535, 'SN'), (143599, 'SC'),
    (143600, 'SL'), (143464, 'SG'), (143496, 'SK'), (143499, 'SI'), (143601, 'SB'),
    (143472, 'ZA'), (143466, 'KR'), (143454, 'ES'), (143486, 'LK'), (143548, 'KN'),
    (143549, 'LC'), (143550, 'VC'), (143554, 'SR'), (143602, 'SZ'), (143456, 'SE'),
    (143459, 'CH'), (143470, 'TW'), (143603, 'TJ'), (143572, 'TZ'), (143475, 'TH'),
    (143551, 'TT'), (143536, 'TN'), (143480, 'TR'), (143604, 'TM'), (143552, 'TC'),
    (143537, 'UG'), (143492, 'UA'), (143481, 'AE'), (143444, 'GB'), (143441, 'US'),
    (143514, 'UY'), (143566, 'UZ'), (143502, 'VE'), (143471, 'VN'), (143571, 'YE'),
    (143605, 'ZW')]
new_ios_list=[(x[0],x[1],"apple-store") for x in IOS_STORE_COUNTRY_MAPPING ]
ANDROID_STORE_COUNTRY_MAPPING = [
    (17, 'AR'), (1, 'AU'), (35, 'AT'), (61, 'AZ'), (11, 'BE'), (18, 'BR'), (47, 'BG'),
    (2, 'CA'), (13, 'CL'), (3, 'CN'), (52, 'CO'), (64, 'CR'), (80, 'HR'), (36, 'CZ'),
    (38, 'DK'), (62, 'EC'), (33, 'EG'), (20, 'FI'), (6, 'FR'), (4, 'DE'), (46, 'GR'),
    (16, 'HK'), (37, 'HU'), (19, 'IN'), (21, 'ID'), (39, 'IE'), (40, 'IL'), (8, 'IT'),
    (9, 'JP'), (53, 'KZ'), (95, 'KE'), (50, 'KW'), (86, 'LV'), (65, 'LB'), (78, 'LT'),
    (24, 'MY'), (26, 'MX'), (23, 'NL'), (41, 'NZ'), (74, 'NG'), (42, 'NO'), (54, 'PK'),
    (56, 'PE'), (31, 'PH'), (28, 'PL'), (43, 'PT'), (84, 'PR'), (73, 'QA'), (44, 'RO'),
    (22, 'RU'), (51, 'SA'), (32, 'SG'), (45, 'SK'), (14, 'ZA'), (27, 'KR'), (5, 'ES'),
    (34, 'SE'), (12, 'CH'), (30, 'TW'), (29, 'TH'), (25, 'TR'), (48, 'UA'), (49, 'AE'),
    (7, 'GB'), (10, 'US'), (15, 'VN'), (1000, 'WW')
]
new_gp_list=[(x[0],x[1],"google-play") for x in ANDROID_STORE_COUNTRY_MAPPING ]

total_list = new_ios_list + new_gp_list
temp_df = spark.createDataFrame(total_list,["auto_pipeline_id","auto_pipeline_country_code","auto_pipeline_market"]).cache()

In [0]:


ANDROID_CATEGORIES = [
    (1, 400000), (2, 400001), (3, 400022), (4, 400023), (5, 400024),
    (6, 400008), (7, 400011), (8, 400014), (9, 400017), (10, 400020),
    (11, 400025), (12, 400030), (13, 400031), (14, 400032), (15, 400033),
    (16, 400035), (17, 400036), (18, 400038), (19, 400040), (20, 400042),
    (21, 400043), (22, 400044), (23, 400058), (24, 400046), (25, 400047),
    (26, 400048), (27, 400050), (28, 400051), (29, 400052), (30, 400053),
    (31, 400054), (32, 400055), (33, 400056), (34, 400045), (35, 400057),
    (36, 400059), (37, 400060), (38, 400002), (39, 400003), (40, 400021),
    (41, 400004), (42, 400005), (43, 400006), (44, 400007), (46, 400009),
    (47, 400010), (48, 400012), (49, 400013), (51, 400015), (52, 400016),
    (54, 400018), (55, 400019), (56, 400061), (57, 400063), (58, 400064),
    (59, 400065), (60, 400062), (61, 400066), (62, 400067), (63, 400068),
    (64, 400069), (65, 400070), (66, 400026), (67, 400027), (68, 400041),
    (69, 400028), (70, 400029), (71, 400034), (72, 400037), (73, 400039),
    (75, 400049)
]


IOS_CATEGORIES = [
    (36, 100000), (100, 100021), (360, 100030), (361, 100031), (362, 100032),
    (363, 100033), (6000, 100023), (6001, 100077), (6002, 100076), (6003, 100075),
    (6004, 100073), (6005, 100072), (6006, 100070), (6007, 100069), (6008, 100068),
    (6009, 100067), (6010, 100066), (6011, 100065), (6012, 100034), (6013, 100029),
    (6014, 100001), (6015, 100027), (6016, 100026), (6017, 100025), (6018, 100022),
    (6020, 100064), (6021, 100035), (6022, 100024), (6023, 100028), (6024, 100071),
    (6025, 100074), (7001, 100002), (7002, 100003), (7003, 100004), (7004, 100005),
    (7005, 100006), (7006, 100007), (7007, 100008), (7008, 100009), (7009, 100010),
    (7010, 100011), (7011, 100012), (7012, 100013), (7013, 100014), (7014, 100015),
    (7015, 100016), (7016, 100017), (7017, 100018), (7018, 100019), (7019, 100020),
    (13001, 100053), (13002, 100046), (13003, 100049), (13004, 100054), (13005, 100060),
    (13006, 100037), (13007, 100036), (13008, 100038), (13009, 100039), (13010, 100040),
    (13011, 100041), (13012, 100042), (13013, 100043), (13014, 100044), (13015, 100045),
    (13017, 100047), (13018, 100048), (13019, 100050), (13020, 100051), (13021, 100052),
    (13023, 100055), (13024, 100056), (13025, 100057), (13026, 100058), (13027, 100059),
    (13028, 100061), (13029, 100062), (13030, 100063)
]


new_category_ios_list=[(x[0],x[1],"ios") for x in IOS_CATEGORIES ]
new_category_gp_list=[(x[0],x[1],"android") for x in ANDROID_CATEGORIES ]

total_list = new_category_ios_list + new_category_gp_list

temp_df = spark.createDataFrame(total_list,["auto_pipeline_legacy_category_id","auto_pipeline_category_id","auto_pipeline_market"]).cache()

In [0]:
%%sh
aws s3 ls s3://prod_appannie_uniform_public_app_store_data/


In [0]:
%%sh
